In [5]:
import sys, json, time
from threading import Thread, Condition, Event
from psonic import *
sys.path.append("./repo/piplay")
from weather_pilock.SenseHat_Data import Sensor
from pysonic.play_weather import Notes, Rates, rescale
from helpers.helpers import *

golden_ratio = golden()


In [6]:
condition = Condition()
stop_event = Event()
rates = Rates()
senseHat = Sensor()

def live_producer():
    sleep(rates.seconds_per_beat)
    pass
def live_loop_producer(condition, stop_event):
    while not stop_event.is_set():
        with condition:
            condition.notifyAll() #Message to threads
        live_producer()

def play_temp():
    pass
def live_loop_temp(condition, stop_event):
     while not stop_event.is_set():
        with condition:
            condition.wait() #Message to threads
        play_temp()

def play_pressure():
    pass    
def live_loop_pressure(condition, stop_event):
    while not stop_event.is_set():
        with condition:
            condition.wait() #Wait for message
        play_pressure()

def play_humidity():
    pass
def live_loop_humidity(condition, stop_event):
    while not stop_event.is_set():
        with condition:
            condition.wait() #Wait for message
        play_humidity()


live_thread_producer = Thread(name="producer", target=live_loop_producer, args=(condition, stop_event))
live_thread_temp = Thread(name="consumer_temp", target=live_loop_temp, args=(condition, stop_event))
live_thread_pressure = Thread(name="consumer_pressure", target=live_loop_pressure, args=(condition, stop_event))
live_thread_humidity = Thread(name="consumer_humidity", target=live_loop_humidity, args=(condition, stop_event))

live_thread_producer.start()
live_thread_temp.start()
live_thread_pressure.start()
live_thread_humidity.start()



In [7]:
def play_weather(senseHat):
    while True:
        senseHat.touch()
        temp = Notes([rescale(senseHat.sense.temperature_from_humidity), 
                 rescale(senseHat.sense.temperature_from_pressure)], 
                     attack = 0.5, 
                     amp = 0.5
                    )
        pressure = Notes([rescale(senseHat.sense.pressure) / golden_ratio],
                         attack = 0.5, 
                         amp = 0.5
                        )
        humidity = Notes([rescale(senseHat.sense.humidity) / (golden_ratio ** 2)],
                         attack = 0.5, 
                         amp = 0.5
                        )
        
        global play_temp
        def play_temp():
            play(temp.notes, attack=temp.attack, decay=temp.decay, sustain=temp.sustain, release=temp.release, amp=temp.amp)
            pass
    
        global play_pressure
        def play_pressure():
            play(pressure.notes, attack=pressure.attack, decay=pressure.decay, sustain=pressure.sustain, release=pressure.release, amp=pressure.amp)
            pass
    
        global play_humidity
        def play_humidity():
            play(humidity.notes, attack=humidity.attack, decay=humidity.decay, sustain=humidity.sustain, release=humidity.release, amp=humidity.amp)
            pass

In [8]:
play_weather(senseHat)

KeyboardInterrupt: 

In [9]:
stop_event.set()